In [2]:
import lsdtopytools as lsd, glob

/sciclone/home/ntlewis/Nick-Lewis-Research/.conda/lib/python3.8/site-packages/lsdtopytools/geoconvtools.py:79: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython = False)
/sciclone/home/ntlewis/Nick-Lewis-Research/.conda/lib/python3.8/site-packages/lsdtopytools/numba_tools.py:73: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit(nopython=False,

In [3]:
names = glob.glob('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis/reproj/*.tiff')
names = [path.split(sep='/')[-1] for path in names]
names

['Left_Fork_Sandy_Creek-Sandy_Creek_reproj.tiff',
 'Middle_Little_Kanawha_River_reproj.tiff',
 'Mill_Creek_reproj.tiff',
 'Oldtown_Creek-Ohio_River_reproj.tiff',
 'Upper_West_Fork_River_reproj.tiff',
 'Knapp_Creek_reproj.tiff',
 'Laurel_Creek-Cherry_River_reproj.tiff',
 'Hominy_Creek_reproj.tiff',
 'Laurel_Creek_reproj.tiff',
 'Williams_River_reproj.tiff',
 'Headwaters_Gauley_River_reproj.tiff',
 'Shorts_Creek-New_River_reproj.tiff',
 'Dumps_Creek-Clinch_River_reproj.tiff',
 'Laurel_Creek-North_Fork_Holston_River_reproj.tiff',
 'Big_Cedar_Creek-Clinch_River_reproj.tiff',
 'Swords_Creek-Clinch_River_reproj.tiff',
 'Bluestone_Lake-New_River_reproj.tiff',
 'Johns_Creek_reproj.tiff',
 'Knox_Creek-Tug_Fork_reproj.tiff',
 'Middle_Jackson_River_reproj.tiff',
 'Second_Creek_reproj.tiff',
 'East_River-New_River_reproj.tiff',
 'Bluestone_River_reproj.tiff',
 'Hunting_Camp_Creek-Wolf_Creek_reproj.tiff',
 'Clear_Fork-Wolf_Creek_reproj.tiff',
 'Sinking_Creek-New_River_reproj.tiff',
 'Holly_River_re

In [4]:
def make_ksn_df(dem_name, path='/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis/reproj/'):
    name = dem_name[:-5]
    mydem = lsd.LSDDEM(path, dem_name)
    print('Raster Loaded')
    mydem.PreProcessing()
    print('PreProcessed')
    mydem.CommonFlowRoutines()
    print('Flow routines got')
    mydem.ExtractRiverNetwork()
    print('River Network Extracted')
    mydem.DefineCatchment(method='main_basin')
    print('Catchment defined')
    mydem.GenerateChi()
    print('Chi generated')
    mydem.ksn_MuddEtAl2014(target_nodes=70, n_iterations=60, skip=1, minimum_segment_length=10, sigma=2,  nthreads = 1, reload_if_same = False)
    mydem.knickpoint_extraction(lambda_TVD = "auto", combining_window = 30, window_stepped = 80, n_std_dev = 7)
    knickpoint_df = mydem.df_knickpoint
    knickpoint_df.to_csv(f'/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis/ksn_csvs/{name}_knickpoints.csv')

In [5]:
from concurrent.futures import ThreadPoolExecutor
import os
#os.mkdir('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis/ksn_csvs')
for name in names:
    make_ksn_df(name)

Loading the raster from file: /sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis/reproj/Left_Fork_Sandy_Creek-Sandy_Creek_reproj.tiff
LOADING TOOK 0.014983177185058594
I am recasting your nodata values to -9999 (standard LSDTT)
PREPROC TOOK 0.0005748271942138672
Alright, let me summon control upon the c++ code ...
Got it.
INGESTINGINTO CPP TOOK 0.0023391246795654297
I am an empty LSDJunctionNetwork
Nrows:849 and ncols:954
out Nrows:849 and out ncols:954

I am an empty LSDJunctionNetwork
Yes, yes, we know. Ignore the above depressing debugging text, that's perfectly normal to feel empty sometimes.
FINALISATION TOOK 0.00019598007202148438
lsdtopytools is now ready to roll!
Raster Loaded
Carving: implementation of Lindsay (2016) DOI: 10.1002/hyp.10648
Filling: implementation of Wang and Liu (2006): https://doi.org/10.1080/13658810500433453
Processing...
I am going to carve/breach your depressions in order to force flow paths.
I am using an algorithm from Lindsa